# 对新学的内容的测试

In [1]:
import sys
sys.path.append('../..')

import numpy as np
import matrixslow as ms

# 生成数据

male_heights = np.random.normal(171, 6, 500)
female_heights = np.random.normal(158, 5, 500)

male_weights = np.random.normal(70, 10, 500)
female_weights = np.random.normal(57, 8, 500)

male_bfrs = np.random.normal(16, 2, 500)
female_bfrs = np.random.normal(22, 2, 500)

In [2]:
male_labels = [1] * 500
female_labels = [-1] * 500

#male_labels

In [3]:
#组合数据

train_set = np.array([np.concatenate((male_heights, female_heights)),
                      np.concatenate((male_weights, female_weights)),
                      np.concatenate((male_bfrs, female_bfrs)),
                      np.concatenate((male_labels, female_labels))]).T


np.random.shuffle(train_set)
# 由输出可见array中第一个元素为一个样本
train_set

array([[171.21966958,  74.52480776,  13.70133268,   1.        ],
       [163.45682221,  61.72999744,  22.3859531 ,  -1.        ],
       [169.64023457,  69.46637363,  12.58778633,   1.        ],
       ...,
       [181.40011955,  69.76301993,  15.63583609,   1.        ],
       [153.19605242,  66.96941103,  18.68954512,  -1.        ],
       [159.39800141,  59.73767205,  17.57935046,  -1.        ]])

In [4]:
# 定义变量

x = ms.core.Variable(dim=(3,1), init=False, trainable=False)
w = ms.core.Variable(dim=(1,3), init=True, trainable=True)
b = ms.core.Variable(dim=(1,1), init=True, trainable=True)
label = ms.core.Variable(dim=(1,1), init=False, trainable=False)

定义节点，无论是Add、Step还是loss function，其实都是node，定义后调用基类node的__init__构造函数，制造出父子双向关系并放入计算图中；而这些都是定义，真正数据的计算发生在前向传播和后向传播中，前向传播forward()递归调用节点的compute()，根据不同node调用不同compute的实现。
节点的定义可以嵌套调用。

In [5]:
# 定义计算图
output = ms.ops.Add(ms.ops.MatMul(w,x), b)
predict = ms.ops.Step(output) #激活函数

# 定义损失函数
loss = ms.ops.loss.PerceptionLoss(ms.ops.MatMul(output, label))

# 定义学习率
learning_rate = 0.0001

In [6]:
features = np.mat(train_set[1,:-1]).T
x.set_value(features)
        
l = np.mat(train_set[1,-1])
label.set_value(l)
        
# 前向传播计算损失值，以作为之后参数训练的结果节点
loss.forward()

w.backward(loss)
b.backward(loss)

loss.value

matrix([[0.20615738]])

In [9]:
w.jacobi.T
w.shape()
#w.jacobi.T.reshape(w.shape())

(1, 3)

In [8]:
# 开始迭代50个epoch
for epoch in range(50):
    # 循环训练样本
    for i in range(len(train_set)):
        # 初始化
        features = np.mat(train_set[i,:-1]).T
        x.set_value(features)
        
        l = np.mat(train_set[i,-1])
        label.set_value(l)
        
        # 前向传播计算损失值，以作为之后参数训练的结果节点
        loss.forward()
        
        # 反向传播，更新参数
        w.backward(loss)
        b.backward(loss)
        
        w.set_value(w.value - learning_rate*w.jacobi.T.reshape(w.shape()))
        b.set_value(b.value - learning_rate*b.jacobi.T.reshape(b.shape()))
        
        ms.default_graph.clear_jacobi()
    
    #epoch结束后评价准确率
    pred = []
    
    #训练完参数后对predict进行向前传播，分析正确率
    for i in range(len(train_set)):
        features = np.mat(train_set[i, :-1]).T
        x.set_value(features)
        
        l = np.mat(train_set[i,-1])
        label.set_value(l)
        
        # 前向传播计算pred结果并保存到数组中
        predict.forward()
        pred.append(predict.value[0,0])
        
    pred = np.array(pred)*2 -1
    
    accuracy = (train_set[:,-1] == pred).astype(np.int).sum() / len(train_set)
    print("epoch: {:d}, accuracy: {:.3f}".format(epoch+1, accuracy))

'\n# 开始迭代50个epoch\nfor epoch in range(50):\n    # 循环训练样本\n    for i in range(len(train_set)):\n        # 初始化\n        features = np.mat(train_set[i,:-1]).T\n        x.set_value(features)\n        \n        l = np.mat(train_set[i,-1])\n        label.set_value(l)\n        \n        # 前向传播计算损失值，以作为之后参数训练的结果节点\n        loss.forward()\n        \n        # 反向传播，更新参数\n        w.backward(loss)\n        b.backward(loss)\n        \n        w.set_value(w.value - learning_rate*w.jacobi.T.reshape(w.shape()))\n        b.set_value(b.value - learning_rate*b.jacobi.T.reshape(b.shape()))\n        \n        ms.default_graph.clear_jacobi()\n    \n    #epoch结束后评价准确率\n    pred = []\n    \n    #训练完参数后对predict进行向前传播，分析正确率\n    for i in range(len(train_set)):\n        features = np.mat(train_set[i, :-1]).T\n        x.set_value(features)\n        \n        l = np.mat(train_set[i,-1])\n        label.set_value(l)\n        \n        # 前向传播计算pred结果并保存到数组中\n        predict.forward()\n        pred.append(predict.valu

# 总结

在概念上厘清，在实现上支持，在名词上谦虚。